The data used here can be found [here](https://www.pxweb.bfs.admin.ch/Selection.aspx?px_language=fr&px_db=px-x-0702000000_104&px_tableid=px-x-0702000000_104\px-x-0702000000_104.px&px_type=PX).

We gathered the data from 2013-2015 (2016) is not available. However, the names of the municipalities are those from the year 2016 (or later). In ordrer to have some data for the right municipality names, we assume that the ratio cow/inhabitant doesn't change between the municipalities that merge together.

In [1]:
# imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import show
import re

In [12]:
df = pd.read_csv('data/data_to_be_processed/cows.csv', encoding='latin_1', skiprows=2, delimiter=';')

In [13]:
df = df.drop(['Système d\'exploitation'], axis=1)
df.columns = ['year', 'commune', 'cows']
df.head()

,year,commune,cows
0,2013,- Suisse,1557474
1,2013,- Zürich,93233
2,2013,>> Bezirk Affoltern,9315
3,2013,......0001 Aeugst am Albis,622
4,2013,......0002 Affoltern am Albis,596


### Preprocessing

In [14]:
#remove districts
df = df.loc[(df.commune.str.contains('>>') == False)]
#remove cantons
df = df.loc[(df.commune.str.contains('- ') == False)]
#remove swiss total
df = df[df['commune'] != '- Suisse']
#rewrite commune name
df['id'], df['commune'] = df['commune'].str.split(' ', 1).str
regex = re.compile(r"\.{6}")
df['id'].replace(to_replace=regex, value='', inplace=True, regex=True)
df['id'] = df['id'].astype(int)
df.head()

,year,commune,cows,id
3,2013,Aeugst am Albis,622,1
4,2013,Affoltern am Albis,596,2
5,2013,Bonstetten,314,3
6,2013,Hausen am Albis,1368,4
7,2013,Hedingen,312,5


In [15]:
# tmp = df[df['year'] == 2015]
# tmp = tmp.drop('year', axis=1)
tmp = df

In [16]:
# def create_2016_data(data):
f = pd.read_csv("data/fusion_mappings.csv")
regex = re.compile(r"\d\d\.\d\d\.")
f['date'].replace(to_replace=regex, value='', inplace=True, regex=True)
fu = pd.merge(f, tmp, left_on='post_name', right_on='commune', how='right')
# fu['commune'] = fu['post_name'].fillna(fu['commune'])
# fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1, inplace=True)
# fu = fu.groupby(['commune', 'id', 'nationality', 'age'], as_index=False).sum()
#     return fu

In [17]:
fu[fu.date.astype(float) < fu.year] # commune = pre_name, id = pr_id

#final: compute_ratio_id, id, commune, cows, year
#compute_ratio_id = id of municipality of current name (column 'id')
#id, commune : id of municipality adjusted by the years (pre_id if mun has merged ( date > year), else id)
#year = column year
fu['compute_ratio_id'] = fu['id']
#check if equality is in the right direction
fu['id'] = fu.apply((lambda x: x.pre_id if (float(x.date) > x.year) else x.id ), axis=1 )
fu['commune'] = fu.apply((lambda x: x.pre_name if (float(x.date) > x.year) else x.commune ), axis=1 )

In [18]:
fu.head(15)

,pre_id,pre_name,post_id,post_name,date,year,commune,cows,id,compute_ratio_id
0,308.0,Niederried bei Kallnach,304.0,Kallnach,2013,2013,Kallnach,1266,304.0,304
1,308.0,Niederried bei Kallnach,304.0,Kallnach,2013,2014,Kallnach,1440,304.0,304
2,308.0,Niederried bei Kallnach,304.0,Kallnach,2013,2015,Kallnach,1292,304.0,304
3,548.0,Ruppoldsried,310.0,Rapperswil (BE),2013,2013,Rapperswil (BE),1716,310.0,310
4,532.0,Bangerten,310.0,Rapperswil (BE),2016,2013,Bangerten,1716,532.0,310
5,548.0,Ruppoldsried,310.0,Rapperswil (BE),2013,2014,Rapperswil (BE),1679,310.0,310
6,532.0,Bangerten,310.0,Rapperswil (BE),2016,2014,Bangerten,1679,532.0,310
7,548.0,Ruppoldsried,310.0,Rapperswil (BE),2013,2015,Rapperswil (BE),1695,310.0,310
8,532.0,Bangerten,310.0,Rapperswil (BE),2016,2015,Bangerten,1695,532.0,310
9,631.0,Trimstein,616.0,Münsingen,2013,2013,Münsingen,862,616.0,616


In [19]:
fu = fu.drop(['pre_id', 'pre_name', 'post_id', 'post_name', 'date'], axis=1)

In [22]:
fu['id'] = fu['id'].astype(int)
fu.to_csv('data/data_to_be_processed/cows_processed.csv', index=False)
fu.head()

,year,commune,cows,id,compute_ratio_id
0,2013,Kallnach,1266,304,304
1,2014,Kallnach,1440,304,304
2,2015,Kallnach,1292,304,304
3,2013,Rapperswil (BE),1716,310,310
4,2013,Bangerten,1716,532,310
